In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
titanic = pd.read_csv('/kaggle/input/titanic/train.csv')

df_train = pd.DataFrame(titanic)


pd.set_option('display.max_rows', 4)  # Show 4 rows
pd.set_option('display.max_columns', None)  # Show all columns


df_train


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
...,...,...,...,...,...,...,...,...,...,...,...,...
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.7500,NaN,Q


In [3]:
df_train.nunique()

PassengerId    891
Survived         2
              ... 
Cabin          147
Embarked         3
Length: 12, dtype: int64

In [4]:
print(df_train['Age'].value_counts())

24.0    30
22.0    27
        ..
23.5     1
74.0     1
Name: Age, Length: 88, dtype: int64


# Dataset 1

Dropping     Cabin, Ticket, Name.

In [5]:
pd.set_option('display.max_rows', 4)  # Show all rows

df_train1 = df_train

df_train1.drop(columns = ['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)

df_train1


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
...,...,...,...,...,...,...,...,...,...
889,890,1,1,male,26.0,0,0,30.0000,C
890,891,0,3,male,32.0,0,0,7.7500,Q


In [6]:
df_train1['Embarked'].fillna('S', inplace=True)

In [7]:
df_train1['Sex'] = df_train1['Sex'].map({'male': 1, 'female': 2})    # male=1 female=2

df_train1

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,S
1,2,1,1,2,38.0,1,0,71.2833,C
...,...,...,...,...,...,...,...,...,...
889,890,1,1,1,26.0,0,0,30.0000,C
890,891,0,3,1,32.0,0,0,7.7500,Q


In [8]:
df_train1['Embarked'] = df_train1['Embarked'].map({'S': 1, 'C': 2, 'Q': 3})

df_train1   # Two NaN in Embarked column... index 61 829 both survived

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,1
1,2,1,1,2,38.0,1,0,71.2833,2
...,...,...,...,...,...,...,...,...,...
889,890,1,1,1,26.0,0,0,30.0000,2
890,891,0,3,1,32.0,0,0,7.7500,3


In [9]:
#Handling missing age

df_train1_age_Na = df_train1[df_train1['Age'].isna()]

df_train1 = df_train1.dropna(subset=['Age'])

In [10]:
df_train1_age_Na

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,6,0,3,1,NaN,0,0,8.4583,3
17,18,1,2,1,NaN,0,0,13.0000,1
...,...,...,...,...,...,...,...,...,...
878,879,0,3,1,NaN,0,0,7.8958,1
888,889,0,3,2,NaN,1,2,23.4500,1


In [11]:
df_train1_age = df_train1

df_train1['Family'] = df_train1['SibSp'] + df_train1['Parch']

/tmp/ipykernel_313/2467646141.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1['Family'] = df_train1['SibSp'] + df_train1['Parch']


In [12]:
#Model to predict age.

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import GridSearchCV

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [21]:
X_age = df_train1_age.drop(columns = ['Age', 'PassengerId'])
y_age = df_train1_age['Age']

scaler = StandardScaler()
#scaler = MinMaxScaler()


X_age_scaled = scaler.fit_transform(X_age)

y_age_scaled = scaler.fit_transform(y_age.values.reshape(-1, 1)).ravel()
scaler_params = (scaler.mean_, scaler.scale_)
X_train, X_test, y_train, y_test = train_test_split(X_age_scaled, y_age_scaled, test_size = 0.2,random_state = 28)

In [ ]:
model = RandomForestRegressor(n_estimators=50, random_state=28)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


'''
# Reshape y_pred_scaled and y_test to 2D arrays
y_pred = y_pred.reshape(-1, 1)
y_test_scaled = y_test.reshape(-1, 1)

# Inverse transform the predicted 'Age' values and the true 'Age' values to the original scale
y_pred = scaler.inverse_transform(y_pred).ravel()
y_true = scaler.inverse_transform(y_test_scaled).ravel()
'''


y_pred = y_pred.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)


y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)


mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

In [14]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a RandomForestRegressor instance
rf = RandomForestRegressor(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to your data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_rf = grid_search.best_estimator_

In [ ]:
X_age = df_train1_age.drop(columns = ['Age', 'PassengerId'])
y_age = df_train1_age['Age']

scaler = StandardScaler()
#scaler = MinMaxScaler()


X_age_scaled = scaler.fit_transform(X_age)
y_age_scaled = scaler.fit_transform(y_age.values.reshape(-1, 1)).ravel()

X_train, X_test, y_train, y_test = train_test_split(X_age_scaled, y_age_scaled, test_size = 0.2,random_state = 28)

model = RandomForestRegressor(n_estimators=50, random_state=28)
model.fit(X_train, y_train)




In [ ]:
y_pred.shape

In [ ]:
y_pred = best_rf.predict(X_test)

y_pred = y_pred.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

In [ ]:
y_pred_round = np.round(y_pred)

y_pred_round

In [ ]:
# now for the missing data.
pd.set_option('display.max_rows', 6) 
df_train1_age_Na


In [15]:
df_train1_age_Na['Family'] = df_train1_age_Na['SibSp'] + df_train1_age_Na['Parch']

/tmp/ipykernel_313/2457252628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1_age_Na['Family'] = df_train1_age_Na['SibSp'] + df_train1_age_Na['Parch']


In [ ]:
df_train1_age_Na

In [16]:
X_age = df_train1_age_Na.drop(columns = ['Age', 'PassengerId'])


X_age_scaled = scaler.fit_transform(X_age)

y_pred_na = best_rf.predict(X_age_scaled)


In [22]:
y_pred_na = y_pred_na.reshape(-1, 1)
y_pred_na = scaler.transform(y_pred_na, scaler_params)

In [23]:
y_pred_na_orig = scaler.inverse_transform(y_pred_na, scaler_params)

In [24]:
y_pred_na_orig

array([[-0.30805314],
       [ 0.38651134],
       [ 0.16597752],
       [-0.32591991],
       [-0.61793433],
       [-0.36519681],
       [ 0.06494378],
       [-0.54323598],
       [ 0.10477852],
       [-0.36519681],
       [-0.44717622],
       [-0.32000386],
       [-0.54323598],
       [ 0.00879119],
       [ 0.16575214],
       [ 0.99457886],
       [-1.49756092],
       [-0.36519681],
       [-0.44717622],
       [-0.54323598],
       [-0.44717622],
       [-0.44717622],
       [-0.36519681],
       [-0.15347138],
       [-0.54789001],
       [-0.44717622],
       [-0.24754663],
       [-0.66165308],
       [ 0.27085745],
       [-0.40405419],
       [-0.34136322],
       [-0.8745541 ],
       [-0.32156816],
       [ 0.99457886],
       [-0.70529862],
       [-0.87667636],
       [ 0.37246641],
       [ 0.66504992],
       [-0.53706904],
       [-0.24754663],
       [-0.54323598],
       [-0.8745541 ],
       [-0.29556235],
       [-0.36519681],
       [-0.75321021],
       [-0

In [ ]:
X_age_scaled = X_age_scaled.reshape(-1, 1)
y_pred_na = y_pred.reshape(-1, 1)
X_age_scaled = scaler.inverse_transform(X_age_scaled)
y_pred_na = scaler.inverse_transform(y_pred_na)


In [ ]:
y_pred_na

In [ ]:
'''
y_pred_na = y_pred_na.reshape(-1)  # Reshape back to (177,)
y_pred_na = scaler.inverse_transform(y_pred_na.reshape(-1, 1))  # Apply inverse transform
'''

In [ ]:
#Grid Search:
#Grid search exhaustively searches through a specified set of hyperparameter combinations. Here's how you can use grid search:

#Randomized Search:
#Randomized search explores a random subset of the hyperparameter space. This can be more efficient when the hyperparameter space is large.